[Training code is from this repo of mine](https://github.com/ceshine/finetuning-t5/tree/mt5-classifier-single-token/mnli). Currently my fine-tunes mT5 models still underperforms comparing to the BERT and XLM models.

The mT5-base model was [trained on the MNLI dataset (Enlgish-only)](https://www.kaggle.com/ceshine/preprocess-and-finetune-mt5?scriptVersionId=52689872). The result is slightly better than [the zero-shot one](https://www.kaggle.com/ceshine/mt5-base-mnli-zero-shot).

The embedding matrix in the encoder is frozen when training.

Update in version 8: Use AdaFactor optimizer; update the maxlen trucation scheme; use the Sortish sampler. Train all weights.

In [ ]:
!pip install --upgrade pip
!pip uninstall -y allennlp
!pip install transformers==4.1.1 typer
!pip install -U pytorch-lightning
!pip install https://github.com/veritable-tech/pytorch-lightning-spells/archive/master.zip

In [ ]:
!mkdir -p /src/finetuning-t5
!git clone https://github.com/ceshine/finetuning-t5.git -b master /src/finetuning-t5

In [ ]:
%cd /src/finetuning-t5/mnli

In [ ]:
!mkdir -p data/kaggle
!cp -r /kaggle/input/contradictory-my-dear-watson/* data/kaggle
!ls data/kaggle/

In [ ]:
!mkdir -p data/mt5-base-mnli
!cp -r /kaggle/input/mt5-base-mnli-pretrained/mt5-base_best/* data/mt5-base-mnli
!ls data/mt5-base-mnli/

In [ ]:
!mkdir -p cache/kaggle/
!python preprocess/preprocess_kaggle.py
!python preprocess/tokenize_dataset.py kaggle --tokenizer-name data/mt5-base-mnli

In [ ]:
!SEED=9923 python train.py --t5-model data/mt5-base-mnli --batch-size 64 --grad-accu 1 \
        --epochs 2 --lr 1e-4 --disable-progress-bar --dataset kaggle \
        --max-len 128 --freeze-embeddings --valid-frequency 0.25 

In [ ]:
!ls cache/
!mv cache/tb_logs /kaggle/working
!mv cache/mt5-base-mnli_best /kaggle/working

In [ ]:
!python kaggle_inference.py /kaggle/working/mt5-base-mnli_best
!cp submission.csv /kaggle/working